<div style="text-align: center;">
    <p style="font-size: 50px;">Práctica 2: Residual Neural Networks</p>
</div>

## Parte 1
##### Definimos ResidualBlock especificada en la __figura 1__, que es una capa residual, posteriormente utilizada para crear la red.

<div style="text-align: center;">
    <img src="residualBlock.jpeg" alt="Capa Residual" width="600">
    <p>Figura 1: Capa residual </p>
</div>

In [13]:
import tensorflow as tf
from tensorflow.keras import layers, Model, models

class ResidualBlock(Model):
    def __init__(self, input_channels, output_channels, strides=(1, 1)):
        super(ResidualBlock, self).__init__()

        # Camino principal: BatchNorm → SiLU → Conv2D → BatchNorm → SiLU → Conv2D
        self.bn1 = layers.BatchNormalization()
        self.activation1 = layers.Activation('swish')  # SiLU es 'swish' en Keras
        self.conv1 = layers.Conv2D(
            output_channels, 
            kernel_size=3, 
            strides=strides, 
            padding="same", 
            use_bias=False
        )

        self.bn2 = layers.BatchNormalization()
        self.activation2 = layers.Activation('swish')
        self.conv2 = layers.Conv2D(
            output_channels, 
            kernel_size=3, 
            strides=(1, 1), 
            padding="same", 
            use_bias=False
        )

        # Conexión residual ajustada si los canales no coinciden
        self.adjust_residual = None
        if input_channels != output_channels:
            self.adjust_residual = layers.Conv2D(
                output_channels, 
                kernel_size=1, 
                strides=strides, 
                padding="same", 
                use_bias=False
            )
    
    def call(self, x):
        
        # Camino principal
        x = self.bn1(x)
        x = self.activation1(x)
        residual = x
        x = self.conv1(x)

        x = self.bn2(x)
        x = self.activation2(x)
        x = self.conv2(x)
        
        # Ajustar conexión residual si es necesario
        if self.adjust_residual is not None:  # Caso 2: canales diferentes
            residual = self.adjust_residual(residual)
        
        # Suma residual
        return x + residual  # La suma funciona para ambos casos

##### Definimos la red ResidualNetwork especificada en la __figura 2__.

<div style="text-align: center;">
    <img src="residualNetwork.jpeg" alt="ResidualNetwork" width="500">
    <p>Figura 2: ResidualNetwork </p>
</div>

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

# Crear el modelo secuencial
model = Sequential([
    # Convolución inicial (sin BatchNorm ni activación en esta capa)
    layers.Conv2D(16, kernel_size=3, strides=(1, 1), padding="same", use_bias=False, input_shape=(32, 32, 3)),

    # Grupo 1: 3 bloques residuales con canales 64
    ResidualBlock(16, 64, strides=(1, 1)),
    ResidualBlock(64, 64, strides=(1, 1)),
    ResidualBlock(64, 64, strides=(1, 1)),

    # Grupo 2: 3 bloques residuales con canales 128 (reducción de tamaño en el primer bloque)
    ResidualBlock(64, 128, strides=(2, 2)),  # La reducción de tamaño aquí
    ResidualBlock(128, 128, strides=(1, 1)),
    ResidualBlock(128, 128, strides=(1, 1)),

    # Grupo 3: 3 bloques residuales con canales 256 (reducción de tamaño en el primer bloque)
    ResidualBlock(128, 256, strides=(2, 2)),  # La reducción de tamaño aquí
    ResidualBlock(256, 256, strides=(1, 1)),
    ResidualBlock(256, 256, strides=(1, 1)),
    
    layers.BatchNormalization(),
    layers.Activation('swish'),

    # Promedio global y capa de salida
    layers.GlobalAveragePooling2D(),  # Convierte (8, 8, 256) a un vector de tamaño 256
    layers.Dense(100, activation="softmax")  # CIFAR-100 tiene 100 clases
])

# Resumen del modelo para verificar las dimensiones
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 32, 32, 16)     │           432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_9                │ (None, 32, 32, 64)     │        47,424 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_10               │ (None, 32, 32, 64)     │        74,240 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_11               │ (None, 32, 32, 64)     │        74,240 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_12               │ (None, 16, 16, 128)    │       230,144 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_13               │ (None, 16, 16, 128)    │       295,936 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_14               │ (None, 16, 16, 128)    │       295,936 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_15               │ (None, 8, 8, 256)      │       919,040 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_16               │ (None, 8, 8, 256)      │     1,181,696 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_17               │ (None, 8, 8, 256)      │     1,181,696 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_37 (Activation)      │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        25,700 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,327,508 (16.51 MB)

 Trainable params: 4,322,100 (16.49 MB)

 Non-trainable params: 5,408 (21.12 KB)

## Parte 2
##### Para cargar los pesos de la red entrenada utilizaremos la siguiente función.

In [15]:
import pickle

def load_weights(model, weight_file):
    # Cargar los pesos desde el archivo utilizando pickle
    with open(weight_file, 'rb') as f:
        weights = pickle.load(f)

    # Obtener todas las variables entrenables y no entrenables del modelo
    all_vars = model.trainable_weights + model.non_trainable_weights

    # Crear una lista de las variables y sus correspondientes pesos del archivo
    weight_list = [(x, weights[x]) for x in sorted(weights.keys())]
    weights = {}

    # Iterar sobre todas las variables del modelo
    for i, var in enumerate(all_vars):
        # Obtener el nombre de la capa de la variable
        aux = var.path.split('/')[-2:]
        classname = '_'.join(aux[0].split('_')[:-1])
        name = aux[1]

        assigned = False
        
        # Buscar el peso que corresponde a la variable en la lista de pesos
        for j, (key, value) in enumerate(weight_list):
            if classname in key and name in key:
                try:
                    # Asignar el peso a la variable
                    all_vars[i].assign(value)
                    print(f'Asignando {key} a {var.name}')
                except:
                    continue
                print('assinging', key, 'to', var.path)
                del weight_list[j]
                assigned = True
                break

        # Si no se pudo asignar el peso, lanzar una excepción
        if not assigned:
            raise Exception(var.path + ' cannot be loaded')

#####  Comprobamos que la precisión del modelo en CIFAR-100 es superior al 69 %. Para ello, preprocesamos el dataset y cargamos los pesos de ResidualNetwork.

In [ ]:
from tensorflow.keras.datasets import cifar100, cifar10
from tensorflow.keras.utils import to_categorical
import os

# Cargar CIFAR-100
(x_train_100, y_train_100), (x_test_100, y_test_100) = cifar100.load_data()

# Normalizar las imágenes a [-1, 1]
x_train_100 = (x_train_100 / 127.5) - 1  # Escalar a [-1, 1]
x_test_100 = (x_test_100 / 127.5) - 1   # Escalar a [-1, 1]

# Convertir las etiquetas a formato one-hot
y_train_100 = to_categorical(y_train_100, 100)  # 100 clases en CIFAR-100
y_test_100 = to_categorical(y_test_100, 100)

# Compilar el modelo
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Ruta del archivo de pesos
path_to_weights = "/kaggle/input/pickel/p3_model_weights.pkl"
# path_to_weights = os.path.join(".", "p3
#_model_weights.pkl")
# Cargar los pesos en el modelo usando la función `load_weights`
load_weights(model, path_to_weights)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(x_test_100, y_test_100, verbose=2)

# Mostrar la precisión
print(f"Precisión en el conjunto de prueba: {test_acc * 100:.2f}%")

Asignando conv2d_52/kernel a kernel
assinging conv2d_52/kernel to sequential_3/conv2d_22/kernel
Asignando batch_normalization_38/gamma a gamma
assinging batch_normalization_38/gamma to sequential_3/residual_block_9/batch_normalization_19/gamma
Asignando batch_normalization_38/beta a beta
assinging batch_normalization_38/beta to sequential_3/residual_block_9/batch_normalization_19/beta
Asignando conv2d_54/kernel a kernel
assinging conv2d_54/kernel to sequential_3/residual_block_9/conv2d_23/kernel
Asignando batch_normalization_39/gamma a gamma
assinging batch_normalization_39/gamma to sequential_3/residual_block_9/batch_normalization_20/gamma
Asignando batch_normalization_39/beta a beta
assinging batch_normalization_39/beta to sequential_3/residual_block_9/batch_normalization_20/beta
Asignando conv2d_55/kernel a kernel
assinging conv2d_55/kernel to sequential_3/residual_block_9/conv2d_24/kernel
Asignando conv2d_53/kernel a kernel
assinging conv2d_53/kernel to sequential_3/residual_block_

## Parte 3
##### Entrenamos mediante la técnica de fine-tuning, sobre el dataset CIFAR-10, manteniendo fijos los pesos de la red preentrenada.

Hemos declarado data augmentation como un modelo de keras que rotará las imagenes horizontalmente, le aplicará una pequeña rotación aleatoria y un zoom aleatorio. Si hemos puesto esto, y no hemos añadido cosas extra como una rotación vertical, es porque a la hora de modificar las imagenes creadas, hemos buscado que sigan siendo coherentes con los datos a entrenar (no tiene sentido que  el modelo entrene con un avión boca abajo).

In [18]:
# Data Augmentation 
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

# Cargar el conjunto de datos CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalizar las imágenes a rango [-1, 1]
x_train = (x_train / 127.5) - 1
x_test = (x_test / 127.5) - 1 

# Convertir las etiquetas a formato one-hot
y_train = to_categorical(y_train, 10)  # 10 clases en CIFAR-10
y_test = to_categorical(y_test, 10)

En primer lugar hemos realizado un modelo empleando el método fit para comprobar que el bucle desde cero está implementado correctamente.

In [19]:

# Crear un Dataset de TensorFlow para aplicar el Data Augmentation de manera eficiente
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(5000).batch(32).map(lambda x, y: (data_augmentation(x), y)).prefetch(tf.data.AUTOTUNE)

# Test dataset (sin augmentation)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Congelar todas las capas
model.trainable = False
# Descongelar solo la última capa densa
model.layers[-1].trainable = True  # Descongelar la capa Dense(100)

# Crear una nueva lista de capas, excluyendo la última
model_layers = model.layers[:-1]  # Esto elimina la última capa

# Crear un nuevo modelo con las capas restantes
new_model = models.Sequential(model_layers)
# Añadir una nueva capa Dense con 10 neuronas para CIFAR-10
new_model.add(layers.Dense(300, activation='swish'))
new_model.add(layers.Dense(200, activation='swish'))
new_model.add(layers.Dense(100, activation='swish'))
new_model.add(layers.Dense(10, activation='softmax'))

# Resumen del nuevo modelo
new_model.summary()

# Verificamos las capas congeladas 
for layer in new_model.layers:
    if isinstance(layer, layers.Dense):  # Si la capa es una capa densa
        print(f"{layer.name} -> Trainable: {layer.trainable}, Neurons: {layer.units}")
    else:
        print(f"{layer.name} -> {'Trainable' if layer.trainable else 'Frozen'}")

# Compilar el modelo nuevamente 
new_model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

# Entrenamiento con datos de entrenamiento y Data Augmentation aplicado
new_model.fit(train_dataset, epochs=10)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = new_model.evaluate(test_dataset, verbose=2)

# Mostrar la precisión
print(f"Precisión en el conjunto de prueba: {test_acc * 100:.2f}%")


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 32, 32, 16)     │           432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_9                │ (None, 32, 32, 64)     │        47,424 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_10               │ (None, 32, 32, 64)     │        74,240 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_11               │ (None, 32, 32, 64)     │        74,240 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_12               │ (None, 16, 16, 128)    │       230,144 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_13               │ (None, 16, 16, 128)    │       295,936 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_14               │ (None, 16, 16, 128)    │       295,936 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_15               │ (None, 8, 8, 256)      │       919,040 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_16               │ (None, 8, 8, 256)      │     1,181,696 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_17               │ (None, 8, 8, 256)      │     1,181,696 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_37 (Activation)      │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 300)            │        77,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 200)            │        60,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,460,218 (17.01 MB)

 Trainable params: 158,410 (618.79 KB)

 Non-trainable params: 4,301,808 (16.41 MB)

conv2d_22 -> Frozen
residual_block_9 -> Frozen
residual_block_10 -> Frozen
residual_block_11 -> Frozen
residual_block_12 -> Frozen
residual_block_13 -> Frozen
residual_block_14 -> Frozen
residual_block_15 -> Frozen
residual_block_16 -> Frozen
residual_block_17 -> Frozen
batch_normalization_37 -> Frozen
activation_37 -> Frozen
global_average_pooling2d_1 -> Frozen
dense_2 -> Trainable: True, Neurons: 300
dense_3 -> Trainable: True, Neurons: 200
dense_4 -> Trainable: True, Neurons: 100
dense_5 -> Trainable: True, Neurons: 10
Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.6060 - loss: 1.0957
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.6878 - loss: 0.8757
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.7075 - loss: 0.8214
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.7203 - loss: 0.7948
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.7275 - loss: 0.7764
Epoch 6/10
1563/1563 ━━━━━━

Para hacer el bucle de entrenamiento eficiente, primero hemos creado dos modelos separados: frozen_model; siendo el modelo preentrenado original sin la última capa que permanecerá intacto y trainable_model;  que es el modelo que será entrenado en el bucle.
Hemos decidido separarlos en 2, porque así a la hora de calcular el gradiente, si solo le pasamos el modelo entrenable (pero con los inputs siendo procesados por el modelo congelado), ahorramos que calcule un gradiente y mantenga en memoria al modelo que no debe alterar en ningún momento (mayor eficiencia computacional y espacial). También hemos realizado la evalución manual.

In [21]:
# Dividir el modelo en dos partes
frozen_model = Sequential(model.layers[:-1])  # Todas las capas excepto la última

frozen_model.trainable = False  # Congelar las capas

trainable_model = Sequential([
    layers.Dense(300, activation='swish'),  # Capa densa adicional
    layers.Dense(200, activation='swish'),  # Capa densa adicional
    layers.Dense(100, activation='swish'),  # Capa densa adicional
    layers.Dense(10, activation='softmax')  # Salida para CIFAR-10
])

# Configurar parámetros de entrenamiento
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
batch_size = 128
num_epochs = 10

# Crear dataset de entrenamiento con data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Dataset de prueba (sin augmentación)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Bucle de entrenamiento manual
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    epoch_loss = 0
    epoch_accuracy = tf.keras.metrics.CategoricalAccuracy()

    for step, (images, labels) in enumerate(train_dataset):
        # Data augmentation
        augmented_images = data_augmentation(images)
        frozen_output = frozen_model(augmented_images, training=False)  # Bloque congelado

        # Forward pass y cálculo de gradientes
        with tf.GradientTape() as tape:
            predictions = trainable_model(frozen_output, training=True)  # Bloque entrenable
            loss = loss_function(labels, predictions)

        # Backward pass: calcular y aplicar gradientes solo a las capas entrenables
        gradients = tape.gradient(loss, trainable_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, trainable_model.trainable_variables))

        # Actualizar métricas
        epoch_loss += loss.numpy()
        epoch_accuracy.update_state(labels, predictions)

        # Imprimir progreso
        if step % 100 == 0:
            print(f"Step {step}: Loss = {loss.numpy():.4f}, Accuracy = {epoch_accuracy.result().numpy():.4f}")

    # Imprimir métricas por época
    print(f"Epoch {epoch + 1}: Loss = {epoch_loss / len(train_dataset):.4f}, Accuracy = {epoch_accuracy.result().numpy():.4f}")

# Evaluación final en el conjunto de prueba
test_loss, test_accuracy = 0, tf.keras.metrics.CategoricalAccuracy()

for images, labels in test_dataset:
    frozen_output = frozen_model(images, training=False)
    predictions = trainable_model(frozen_output, training=False)
    test_loss += loss_function(labels, predictions).numpy()
    test_accuracy.update_state(labels, predictions)

print(f"Test Loss = {test_loss / len(test_dataset):.4f}, Test Accuracy = {test_accuracy.result().numpy():.4f}")


Epoch 1/10
Step 0: Loss = 2.3177, Accuracy = 0.0547
Step 100: Loss = 0.9445, Accuracy = 0.5685
Step 200: Loss = 1.0307, Accuracy = 0.6107
Step 300: Loss = 0.9692, Accuracy = 0.6281
Epoch 1: Loss = 1.0114, Accuracy = 0.6383
Epoch 2/10
Step 0: Loss = 0.8670, Accuracy = 0.6719
Step 100: Loss = 0.9485, Accuracy = 0.6794
Step 200: Loss = 0.7051, Accuracy = 0.6864
Step 300: Loss = 1.0275, Accuracy = 0.6867
Epoch 2: Loss = 0.8683, Accuracy = 0.6888
Epoch 3/10
Step 0: Loss = 0.7350, Accuracy = 0.7734
Step 100: Loss = 0.8232, Accuracy = 0.7044
Step 200: Loss = 0.8198, Accuracy = 0.7055
Step 300: Loss = 0.6871, Accuracy = 0.7047
Epoch 3: Loss = 0.8234, Accuracy = 0.7050
Epoch 4/10
Step 0: Loss = 0.7179, Accuracy = 0.7344
Step 100: Loss = 0.7417, Accuracy = 0.7150
Step 200: Loss = 0.8147, Accuracy = 0.7149
Step 300: Loss = 0.6927, Accuracy = 0.7149
Epoch 4: Loss = 0.7980, Accuracy = 0.7153
Epoch 5/10
Step 0: Loss = 0.8614, Accuracy = 0.6797
Step 100: Loss = 0.8125, Accuracy = 0.7267
Step 200: Los

### Análisis del resultado del conjunto de test.

El modelo entrenado muestra un rendimiento sólido, con una precisión de test del 76.55% y una pérdida de test de 0.6628, lo que indica una buena capacidad de generalización. Durante el entrenamiento, la precisión aumentó de 5.47% a 75,31%, mientras que la pérdida disminuyó progresivamente desde 2.31 hasta 0.6923, lo que refleja una mejora constante en el aprendizaje. La pequeña diferencia entre las métricas de entrenamiento y test sugiere que el modelo no está sobreajustado, ya que la precisión en el conjunto de test es incluso mayor que en el de entrenamiento. Este comportamiento muestra que el modelo ha aprendido los patrones del conjunto de datos sin memorizar, lo cual es una señal de buena generalización. Sin embargo, se podrían explorar más épocas o técnicas de regularización para mejorar aún más el rendimiento. En general, el modelo ha alcanzado un buen nivel de desempeño en pocas épocas de entrenamiento.

Por último, podemos comprobar que empleando fit y el bucle, los resultados son similares.

## Parte 4
Entrenamos la red por fine tunning pero sin mantener fijos los pesos de la red preentrenada.

En primer lugar hemos realizado un modelo empleando el método fit para comprobar que el bucle desde cero está implementado correctamente.


In [32]:

model.trainable = True

# Congelar solo las capas de BatchNormalization
for layer in model.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False 
    elif isinstance(layer, ResidualBlock):
        if isinstance(layer.bn1, layers.BatchNormalization):
            layer.bn1.trainable = False
        if isinstance(layer.bn2, layers.BatchNormalization):
            layer.bn2.trainable = False
        
# Crear una nueva lista de capas, excluyendo la última
model_layers = model.layers[:-1]  # Esto elimina la última capa

# Crear un nuevo modelo con las capas restantes
new_model = models.Sequential(model_layers)

# Añadir una nueva capa Dense con 10 neuronas para CIFAR-10
new_model.add(layers.Dense(10, activation='softmax'))

# Resumen del nuevo modelo
new_model.summary()

# Verificamos las capas congeladas 
for layer in new_model.layers:
    if isinstance(layer, layers.Dense):  # Si la capa es una capa densa
        print(f"{layer.name} -> Trainable: {layer.trainable}, Neurons: {layer.units}")
    elif isinstance(layer, ResidualBlock):
        print(f"ResidualBlock: {layer.name}")
        if isinstance(layer.bn1, layers.BatchNormalization):
            print(f"  BatchNormalization bn1 -> Trainable: {layer.bn1.trainable}")
        if isinstance(layer.bn2, layers.BatchNormalization):
            print(f"  BatchNormalization bn2 -> Trainable: {layer.bn2.trainable}")
    else:
        print(f"{layer.name} -> {'Trainable' if layer.trainable else 'Frozen'}")


# Compilar el modelo nuevamente
optimizer = tf.keras.optimizers.SGD(momentum=0.9, learning_rate=0.0001 )
new_model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

# Entrenamiento con datos reales
history = new_model.fit(train_dataset, batch_size=32, epochs= 10)

# Evaluar el modelo
test_loss, test_acc = new_model.evaluate(test_dataset, verbose=2)

# Mostrar la precisión
print(f"Precisión en el conjunto de prueba: {test_acc * 100:.2f}%")

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 32, 32, 16)     │           432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_9                │ (None, 32, 32, 64)     │        47,424 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_10               │ (None, 32, 32, 64)     │        74,240 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_11               │ (None, 32, 32, 64)     │        74,240 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_12               │ (None, 16, 16, 128)    │       230,144 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_13               │ (None, 16, 16, 128)    │       295,936 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_14               │ (None, 16, 16, 128)    │       295,936 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_15               │ (None, 8, 8, 256)      │       919,040 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_16               │ (None, 8, 8, 256)      │     1,181,696 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_17               │ (None, 8, 8, 256)      │     1,181,696 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_37 (Activation)      │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,304,378 (16.42 MB)

 Trainable params: 4,293,562 (16.38 MB)

 Non-trainable params: 10,816 (42.25 KB)

conv2d_22 -> Trainable
ResidualBlock: residual_block_9
  BatchNormalization bn1 -> Trainable: False
  BatchNormalization bn2 -> Trainable: False
ResidualBlock: residual_block_10
  BatchNormalization bn1 -> Trainable: False
  BatchNormalization bn2 -> Trainable: False
ResidualBlock: residual_block_11
  BatchNormalization bn1 -> Trainable: False
  BatchNormalization bn2 -> Trainable: False
ResidualBlock: residual_block_12
  BatchNormalization bn1 -> Trainable: False
  BatchNormalization bn2 -> Trainable: False
ResidualBlock: residual_block_13
  BatchNormalization bn1 -> Trainable: False
  BatchNormalization bn2 -> Trainable: False
ResidualBlock: residual_block_14
  BatchNormalization bn1 -> Trainable: False
  BatchNormalization bn2 -> Trainable: False
ResidualBlock: residual_block_15
  BatchNormalization bn1 -> Trainable: False
  BatchNormalization bn2 -> Trainable: False
ResidualBlock: residual_block_16
  BatchNormalization bn1 -> Trainable: False
  BatchNormalization bn2 -> Trainable: 

En este caso, hemos congelado las capas internas de batch normalization de los bloques residuales y además hemos hecho un único modelo para pasárselo al bucle, ya que no hace falta optimizarlo como en la parte 3, porque vamos a entrenarlo todo.

In [29]:
model.trainable = True

# Congelar las capas de BatchNormalization (ya lo tienes implementado correctamente)
for layer in model.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False  # Congelar la capa
    elif isinstance(layer, ResidualBlock):
        if isinstance(layer.bn1, layers.BatchNormalization):
            layer.bn1.trainable = False
        if isinstance(layer.bn2, layers.BatchNormalization):
            layer.bn2.trainable = False
    else:
        layer.trainable = True  # Esto es redundante, pero por si las moscas

# Crear un nuevo modelo eliminando la última capa para agregar una nueva capa de salida
model_layers = model.layers[:-1]
new_model = models.Sequential(model_layers)

# Añadir una nueva capa Dense con 10 neuronas para CIFAR-10
new_model.add(layers.Dense(10, activation='softmax'))

# Resumen del nuevo modelo
new_model.summary()

# Configuración de parámetros de entrenamiento
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=False)  # Usa softmax para la salida
optimizer = tf.keras.optimizers.SGD(momentum=0.9, learning_rate=0.0001)
batch_size = 128
num_epochs = 20

# Crear dataset con data augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Dataset de prueba
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Bucle de entrenamiento manual
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    epoch_loss_metric = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.CategoricalAccuracy()

    for step, (images, labels) in enumerate(train_dataset):
        # Data Augmentation (aplica augmentación si es necesario)
        augmented_images = data_augmentation(images)  # Aplicar augmentación aquí

        # Forward pass y cálculo de gradientes
        with tf.GradientTape() as tape:
            predictions = new_model(augmented_images, training=True)  # Propagación hacia adelante
            loss = loss_function(labels, predictions)  # Cálculo de la pérdida

        # Cálculo de gradientes y actualización de pesos
        trainable_vars = [var for var in new_model.trainable_variables if var.trainable]
        gradients = tape.gradient(loss, trainable_vars)
        optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Actualizar métricas
        epoch_loss_metric.update_state(loss)
        epoch_accuracy.update_state(labels, predictions)

        # Imprimir cada ciertos pasos
        if step % 100 == 0:
            print(f"Step {step}: Loss = {loss.numpy():.4f}, Accuracy = {epoch_accuracy.result().numpy():.4f}")

    # Imprimir resultados al final de cada época
    print(f"Epoch {epoch + 1}: Loss = {epoch_loss / len(train_dataset):.4f}, Accuracy = {epoch_accuracy.result().numpy():.4f}")

# Evaluación en conjunto de prueba
test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.CategoricalAccuracy()

new_model.compile(optimizer=optimizer, 
                        loss=loss_function, 
                        metrics=['accuracy'])

test_loss, test_accuracy = new_model.evaluate(test_dataset)
print(f"Test Loss = {test_loss:.4f}, Test Accuracy = {test_accuracy:.4f}")

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 32, 32, 16)     │           432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_9                │ (None, 32, 32, 64)     │        47,424 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_10               │ (None, 32, 32, 64)     │        74,240 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_11               │ (None, 32, 32, 64)     │        74,240 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_12               │ (None, 16, 16, 128)    │       230,144 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_13               │ (None, 16, 16, 128)    │       295,936 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_14               │ (None, 16, 16, 128)    │       295,936 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_15               │ (None, 8, 8, 256)      │       919,040 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_16               │ (None, 8, 8, 256)      │     1,181,696 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_17               │ (None, 8, 8, 256)      │     1,181,696 │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_37 (Activation)      │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,304,378 (16.42 MB)

 Trainable params: 4,293,562 (16.38 MB)

 Non-trainable params: 10,816 (42.25 KB)


Epoch 1/20
Step 0: Loss = 3.7830, Accuracy = 0.2500
Step 100: Loss = 0.7793, Accuracy = 0.5370
Step 200: Loss = 0.6642, Accuracy = 0.6265
Step 300: Loss = 0.7204, Accuracy = 0.6662
Epoch 1: Loss = 0.8648, Accuracy = 0.6693

Epoch 2/20
Step 0: Loss = 0.7137, Accuracy = 0.7500
Step 100: Loss = 0.7548, Accuracy = 0.7616
Step 200: Loss = 0.4949, Accuracy = 0.7668
Step 300: Loss = 0.5391, Accuracy = 0.7723
Epoch 2: Loss = 0.8648, Accuracy = 0.7728

Epoch 3/20
Step 0: Loss = 0.6298, Accuracy = 0.7500
Step 100: Loss = 0.6039, Accuracy = 0.7894
Step 200: Loss = 0.6176, Accuracy = 0.7926
Step 300: Loss = 0.5615, Accuracy = 0.7939
Epoch 3: Loss = 0.8648, Accuracy = 0.7939

Epoch 4/20
Step 0: Loss = 0.6469, Accuracy = 0.7812
Step 100: Loss = 0.4725, Accuracy = 0.7986
Step 200: Loss = 0.5330, Accuracy = 0.7999
Step 300: Loss = 0.5106, Accuracy = 0.8021
Epoch 4: Loss = 0.8648, Accuracy = 0.8019

Epoch 5/20
Step 0: Loss = 0.5227, Accuracy = 0.8281
Step 100: Loss = 0.5128, Accuracy = 0.8069
Step 200

### Análisis del resultado del conjunto de test.

El modelo entrenado muestra un notable rendimiento en el conjunto de test, con una precisión de test de 85.99% y una pérdida de test de 0.4064, lo que indica un alto nivel de generalización. Durante el entrenamiento, la precisión aumentó de 25.00% a 84.86% y la pérdida disminuyó de 3.78 a 0.8648, mostrando una mejora constante. En las primeras épocas, la precisión creció rápidamente, estabilizándose entorno al 84% hacia el final del entrenamiento. Esta tendencia sugiere que el modelo ha aprendido los patrones de los datos de manera efectiva. La pequeña diferencia entre las métricas de entrenamiento y test indica que el modelo no presenta sobreajuste, ya que las métricas de test son casi tan altas como las de entrenamiento. En resumen, el modelo ha alcanzado un rendimiento excelente y consistente, con solo 20 épocas de entrenamiento, lo que demuestra su capacidad para generalizar bien a datos no vistos.

Por último, podemos comprobar que empleando fit y el bucle, los resultados son similares.